In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchinfo import summary
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import joblib
import pennylane as qml
torch.manual_seed(42)

In [11]:
device = torch.device("cpu")
device

device(type='cpu')

In [12]:
train_data=pd.read_csv('../assets/processed dataset/train dataset.csv')
validation_data=pd.read_csv('../assets/processed dataset/validation dataset.csv')
test_data=pd.read_csv('../assets/processed dataset/test dataset.csv')

In [13]:
#Data scaling
scaler=MinMaxScaler()
normalized_train_data=scaler.fit_transform(train_data)
joblib.dump(scaler, "../assets/scaler.gz") 
print(normalized_train_data[0])
normalized_validation_data=scaler.transform(validation_data)
print(normalized_validation_data[0])
normalized_test_data=scaler.transform(test_data)

[0.61111111 0.19290466 0.20735786 0.21236002]
[0.64285714 0.27937916 0.25139353 0.35296557]


In [14]:
#Prepearing X and Y for training and validation
# MAKING THE SEQUENCE: USING LOOK BACK PERIOD OF 60
seq_length=180
X_train=[]
Y_train=[]
X_validate=[]
Y_validate=[]
X_test=[]
Y_test=[]
for i in range(seq_length,train_data.shape[0]):
    #Data from 0 to 59TH index
    X_train.append(normalized_train_data[i-seq_length:i])
    #T2M AT THE 60TH index
    Y_train.append(normalized_train_data[i])

for i in range(seq_length,validation_data.shape[0]):
    #Data from 0 to 59TH index
    X_validate.append(normalized_validation_data[i-seq_length:i])
    #T2M AT THE 60TH index
    Y_validate.append(normalized_validation_data[i])

for i in range(seq_length,test_data.shape[0]):
    #Data from 0 to 59TH index
    X_test.append(normalized_test_data[i-seq_length:i])
    #T2M AT THE 60TH index
    Y_test.append(normalized_test_data[i])
# converting into numpy arrays
X_train,Y_train=np.array(X_train),np.array(Y_train)
Y_train=Y_train.reshape(-1,4)
print(X_train.shape,Y_train.shape)

X_test,Y_test=np.array(X_test),np.array(Y_test)
Y_test=Y_test.reshape(-1,4)
print(X_test.shape,Y_test.shape)

X_validate,Y_validate=np.array(X_validate),np.array(Y_validate)
Y_validate=Y_validate.reshape(-1,4)
print(X_validate.shape,Y_validate.shape)

X_train = Variable(torch.Tensor(X_train))
Y_train = Variable(torch.Tensor(Y_train))
print(X_train.shape,Y_train.shape)

X_test = Variable(torch.Tensor(X_test))
Y_test = Variable(torch.Tensor(Y_test))
print(X_test.shape,Y_test.shape)

X_validate = Variable(torch.Tensor(X_validate))
Y_validate = Variable(torch.Tensor(Y_validate))
print(X_validate.shape,Y_validate.shape)

(10813, 180, 4) (10813, 4)
(2176, 180, 4) (2176, 4)
(2176, 180, 4) (2176, 4)
torch.Size([10813, 180, 4]) torch.Size([10813, 4])
torch.Size([2176, 180, 4]) torch.Size([2176, 4])
torch.Size([2176, 180, 4]) torch.Size([2176, 4])


In [15]:
#single lstm Network definition
class NETWORK_SINGLE_LSTM(nn.Module):

    def __init__(self,num_classes,input_size,hidden_size,num_layers):
        super(NETWORK_SINGLE_LSTM,self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm=nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True).to(device)
        self.fc=nn.Linear(hidden_size,num_classes)
    
    def forward(self,x):
        output_lstm, (h_n,c_n) = self.lstm(x)
        input_fc=output_lstm[:,-1,:]
        output_fc=self.fc(input_fc).to(device)
        return output_fc

In [16]:
def train_and_save_curve(model, X_train, Y_train, num_epochs, learning_rate, model_name):
# DEFINE OPTIMIZER
    criterion = torch.nn.MSELoss()    # mean-squared error for regression
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Lists to store training losses for plotting the training curve
    train_losses = []
    valid_losses=[]
#batch gradient descent
# TRAIN THE MODEL:
    for epoch in range(1,num_epochs+1):
        model.train()
        outputs = model(X_train.to(device))
        optimizer.zero_grad()
    # obtain the loss function
        loss = criterion(outputs, Y_train.to(device))
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        model.eval()
        with torch.no_grad():
            valid_outputs=model(X_validate.to(device))
            valid_loss=criterion(valid_outputs,Y_validate.to(device))
            valid_losses.append(valid_loss.item())
        if epoch % 10 == 0:
            print("Epoch: %d, training loss: %1.5f, validation loss: %1.5f" % (epoch, loss.item(),valid_loss.item()))
# Plotting the training curve
    epochs=range(1,num_epochs+1)
    st=0;end=num_epochs
    plt.plot( epochs[st:end],train_losses[st:end], label='Training Loss')
    plt.plot(epochs[st:end],valid_losses[st:end], label='Validation Loss')
    plt.title('Epoch vs Loss Curve - ' + model_name)

# plt.plot(epochs, train_losses, label='Training Loss')
# plt.plot(epochs, valid_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('MSE Loss')
    plt.legend()
    plt.savefig('../assets/losses/LSTM/'+model_name + 'LSTM_epoch_vs_loss.png')
    plt.close()
    torch.save(model.state_dict(),'../assets/trainedmodels/LSTM/'+model_name+'LSTM.pt')


In [17]:
# Hyperparameter tuning
num_epochs_list = [10]
learning_rates = [0.01]
hidden_sizes = [1]
input_size=4
num_classes=4
num_layers_list = [1]
# num_epochs_list = [50,100,200]
# learning_rates = [0.01,0.001]
# hidden_sizes = [4,8,16]
# input_size=4
# num_classes=4
# num_layers_list = [1,2,3]

In [18]:

model_names = []
hyperparameters = []
mae_values = []

for num_epochs in num_epochs_list:
    for learning_rate in learning_rates:
        for hidden_size in hidden_sizes:
            for num_layers in num_layers_list:
                model_name = f"LSTM_epochs_{num_epochs}_lr_{learning_rate}_hidden_{hidden_size}_layers_{num_layers}"
                lstm_model = NETWORK_SINGLE_LSTM(num_classes, input_size, hidden_size, num_layers)
                train_and_save_curve(lstm_model, X_train, Y_train, num_epochs, learning_rate, model_name)
                
                # Calculate predictions
                predictions = lstm_model(X_test)
                
                # Inverse transform predictions to get denormalized values
                predictions = predictions.detach().numpy()
                
                # Calculate MAE
                mae = mean_absolute_error(Y_test, predictions)
                
                # Append model name, hyperparameters, and MAE to the lists
                model_names.append(model_name)
                hyperparameters.append((num_epochs, learning_rate, hidden_size, num_layers))
                mae_values.append(mae)

# Create a DataFrame with model names, hyperparameters, and MAE values
results_df = pd.DataFrame({
    'Model': model_names,
    'Num_epochs': [params[0] for params in hyperparameters],
    'Learning_rate': [params[1] for params in hyperparameters],
    'Hidden_size': [params[2] for params in hyperparameters],
    'Num_layers': [params[3] for params in hyperparameters],
    'MAE': mae_values
})

# Save the DataFrame to a CSV file
results_df.to_csv('./results/LSTM/mae_results_with_hyperparametersLSTM.csv', index=False)
